In [0]:
%sql
DROP TABLE IF EXISTS finance_fraudworkspace.silver_managed.transactions_silver;


In [0]:
raw_path = "/Volumes/finance_fraudworkspace/bronze/raw_volume/transactions"
checkpoint_path = "/Volumes/finance_fraudworkspace/bronze/raw_volume/checkpoints/transactions_bronze"
schema_path = "/Volumes/finance_fraudworkspace/bronze/raw_volume/schemas/transactions_bronze"

transactions_stream = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("cloudFiles.schemaLocation", schema_path)
        .option("cloudFiles.includeExistingFiles", "false")
        .load(raw_path)
)

In [0]:
from pyspark.sql.functions import col, upper, to_timestamp, when

transactions_silver_stream = (
    transactions_stream
    
    # Remove records missing keys
    .filter(col("account_id").isNotNull())
    .filter(col("transaction_id").isNotNull())

    .withColumn(
        "transaction_ts",
        to_timestamp(col("transaction_ts"))
    )
    # Remove invalid amounts
    .filter(col("amount").isNotNull())
    .filter(col("amount") > 0)
    
    # Standardize text fields
    .withColumn("channel", upper(col("channel")))
    .withColumn("country", upper(col("country")))
    .withColumn("currency", upper(col("currency")))
)


In [0]:
# WRITE AS DELTA STREAM 
query = (
    transactions_silver_stream.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", checkpoint_path)
        .trigger(availableNow=True)
        .toTable("finance_fraudworkspace.silver_managed.transactions_silver")
)

In [0]:
# DISPLAY LIVE DELTA STREAM
spark.read.table("finance_fraudworkspace.silver_managed.transactions_silver").display()

account_id,amount,channel,country,currency,merchant_id,transaction_id,transaction_ts,_rescued_data
A777,850,ONLINE,US,USD,M009,T9101,2026-01-27T18:00:00Z,null
A777,1250,ONLINE,US,USD,M009,T9102,2026-01-27T18:01:30Z,null
A888,95,POS,US,USD,M010,T9103,2026-01-27T18:02:10Z,null
A777,2100,ONLINE,US,USD,M011,T9104,2026-01-27T18:03:00Z,null
A999,20,POS,US,USD,M012,T9105,2026-01-27T18:04:20Z,null
A777,3300,ONLINE,US,USD,M009,T9106,2026-01-27T18:05:10Z,null
A888,4700,ONLINE,US,USD,M010,T9107,2026-01-27T18:06:40Z,null
A777,5200,ONLINE,US,USD,M009,T9108,2026-01-27T18:07:30Z,null
A999,45,POS,US,USD,M012,T9109,2026-01-27T18:08:15Z,null
A777,7600,ONLINE,US,USD,M011,T9110,2026-01-27T18:09:50Z,null
